In [1]:
import json

with open(r"audios.json", "r", encoding='utf-16') as file:
    audios = json.load(file)

audios[1]

{'model': 'ground_truth.audio',
 'pk': 40,
 'fields': {'music': True,
  'artist': 'Bruno Mars, Anderson.Paak & Silk Sonic',
  'title': 'Smokin Out The Window',
  'filename': 'GG7fLOmlhYg',
  'transcript': "[00:01.30] WAIT A MINUTE THIS LOVE STARTED OUT SO TENDER, SO SWEET \n[00:04.14] BUT NOW SHE GOT ME \n[00:05.10] SMOKIN OUT THE WINDOW \n[00:11.68] MUST HAVE SPENT 35, $45,000 UP IN TIFFANY'S (OH NO) \n[00:17.22] GOT HER BAD --- KIDS RUNNIN ROUND MY WHOLE CRIB LIKE IT'S CHUCK E CHEESE \n[00:21.75] (WHOA, WHOA) \n[00:23.06] PUT ME IN A JAM WITH HER EX-MAN IN THE UFC \n[00:27.66] CAN'T BELIEVE IT (CAN'T BELIEVE IT) \n[00:30.40] I'M IN DISBELIEF \n[00:32.37] THIS ----- GOT ME PAYIN HER RENT, PAYIN FOR TRIPS, \n[00:36.50] DIAMONDS ON HER NECK, DIAMONDS ON HER WRISTS \n[00:38.84] AND HERE I AM ALL ALONE (ALL ALONE) \n[00:42.44] I'M SO COLD, I'M SO COLD, YOU GOT ME OUT HERE \n[00:46.08] SMOKIN OUT THE WINDOW (SMOKING OUT THE WINDOW) \n[00:51.25] SINGING HOW COULD SHE DO THIS TO ME (HOW COUL

In [2]:
with open(r"chunks.json", "r", encoding='utf-16') as file:
    chunks = json.load(file)

chunks[1]

{'model': 'ground_truth.chunk',
 'pk': 5105,
 'fields': {'index': 46,
  'text': ' SMOKIN OUT THE WINDOW ',
  'audio': 40,
  'start_time': 163.3,
  'end_time': 168.47,
  '_image_ids': '[803380, 1006063, 611188, 887084, 1159315, 234984, 1243804, 1159029, 1005721, 611342]',
  '_selected_ids': '[0, 2]',
  'slug': 'chunk-46'}}

In [8]:
for audio in audios:
    print(audio['pk'], audio['fields']['slug'])

filter_audio = [3, 77, 254, 259]

3 bruno-mars-count-on-me
40 bruno-mars-andersonpaak-silk-sonic-smokin-out-the-window
43 america-a-horse-with-no-name
65 weezer-in-the-garage
66 the-beatles-here-comes-the-sun-2019-mix
67 david-bowie-starman-2002-remaster
69 coldplay-viva-la-vida
70 the-beatles-penny-lane
71 electric-light-orchestra-mr-blue-sky
72 phoebe-bridgers-kyoto
77 9imgfqmmufs
254 kanye-west-diamonds-from-sierra-leone-bonus-track
258 radiohead-how-to-disappear-completely
259 radiohead-weird-fishes-arpeggi
261 radiohead-jigsaw-falling-into-place
263 nena-99-luftballons


In [9]:
image_ids = []
for chunk in chunks:
    if chunk['fields']['audio'] not in filter_audio:
        image_ids.extend(json.loads(chunk['fields']['_image_ids']))

image_ids = list(set(image_ids))
len(image_ids)

1847

In [ ]:
import numpy as np
import os

def index_image_paths(folder, imagenet=True):
    try:
        image_paths = []

        if imagenet:
            for synset in os.listdir(folder):
                subfolder = os.path.join(folder, synset)
                if os.path.isdir(subfolder):
                    for image_name in os.listdir(subfolder):
                        image_paths.append(os.path.join(synset, image_name))
        else:
            return os.listdir(folder)
    except Exception as ex:
        print(f"\u001b[31m{type(ex).__name__}: {ex.args}\u001b[0m")

    return image_paths

EXCLUDE = [516621, 653283, 589736, 630260, 590689, 516333, 701639, 538449, 558009, 1017551, 590174, 558027, 590157, 559003, 604023, 572346, 661681, 993150, 1018475, 792565, 1054980, 993647, 515930, 1279994, 1122816, 1122783, 1122318, 559453, 1122669, 1017594, 1122462, 558741, 558533, 559600, 1121668, 1121871, 1122352, 558838, 559166, 559573, 1122659, 1122602, 1147593, 590688, 766038, 589621, ]
IMAGE_PATHS = np.delete(index_image_paths("imagenet-1k"), EXCLUDE)
print(IMAGE_PATHS)

In [67]:
import shutil
import win32api

subset_image_paths = IMAGE_PATHS[image_ids]
dest_path = win32api.GetShortPathName(rf"..\..\..\..\Automatic-Videography-Ground-Truth-Builder\ground_truth_videography_project\i")

for i, ipath in enumerate(subset_image_paths):
    shutil.copyfile(os.path.join("imagenet-1k", ipath), os.path.join(dest_path, f"{image_ids[i]}.jpg"))

In [5]:
def extract_ids(paths):
    return [int(path.split(".")[0]) for path in paths]

IMAGE_PATHS = [""]*1281121
subset_paths = index_image_paths('image_collection', imagenet=False)
subset_ids = extract_ids(subset_paths)
for i, id in enumerate(subset_ids):
    IMAGE_PATHS[id] = subset_paths[i]

IMAGE_PATHS = np.array(IMAGE_PATHS)

In [ ]:
from ground_truth.models import Audio, Chunk
